In [ ]:
!pip install -U mlflow
!pip install -U azureml-mlflow

In [1]:
import mlflow

## Construct AzureML MLFLOW TRACKING URI
def get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace):
    return "azureml://{}.api.azureml.ms/mlflow/v1.0/subscriptions/{}/resourceGroups/{}/providers/Microsoft.MachineLearningServices/workspaces/{}".format(region, subscription_id, resource_group, workspace)

region='eastus' ## example: westus
subscription_id = '062bbb35-45d7-40c6-937f-a43ab3667b0f' ## example: 11111111-1111-1111-1111-111111111111
resource_group = 'rg-dev-geog-eastus' ## example: myresourcegroup
workspace = 'mlw-dev-geog-01' ## example: myworkspacename

MLFLOW_TRACKING_URI = get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace)

#log data locally when running outside azureml
#MLFLOW_TRACKING_URI =""

## Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Make sure the MLflow URI looks something like this: 
## azureml://<REGION>.api.azureml.ms/mlflow/v1.0/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.MachineLearningServices/workspaces/<AML_WORKSPACE_NAME>

print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)


MLFlow Tracking URI: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/062bbb35-45d7-40c6-937f-a43ab3667b0f/resourceGroups/rg-dev-geog-eastus/providers/Microsoft.MachineLearningServices/workspaces/mlw-dev-geog-01


In [2]:
experiment_name = 'experiment_with_mlflow_python'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', experiment_id='8a314941-9201-4060-b57d-36b0e8e1f4c2', lifecycle_stage='active', name='experiment_with_mlflow_python', tags={}>

In [3]:
import os
from random import random
import mlflow

with mlflow.start_run() as mlflow_run:
    mlflow.log_param("hello_param", "world")
    mlflow.log_metric("hello_metric", random())
    os.system(f"echo 'hello world' > helloworld.txt")
    mlflow.log_artifact("helloworld.txt")


In [4]:
# Read metrics after the training
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

# Retrieve run ID for the last run experiement
current_experiment=mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=current_experiment.experiment_id, run_view_type=ViewType.ALL)
run_id = runs.tail(1)["run_id"].tolist()[0]

# Use MlFlow to retrieve the run that was just completed
client = MlflowClient()
finished_mlflow_run = MlflowClient().get_run(run_id)

metrics = finished_mlflow_run.data.metrics
tags = finished_mlflow_run.data.tags
params = finished_mlflow_run.data.params

print(metrics,tags,params)

{'hello_metric': 0.6207762949161376} {'mlflow.user': 'azureuser', 'mlflow.source.name': '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.rootRunId': 'b6121306-c7d0-487b-aabf-873bf7c2f196', 'mlflow.runName': 'mango_whale_qdd31t1n'} {'hello_param': 'world'}


In [5]:
# retrieve artifacts with mlflow
client.list_artifacts(run_id)

[<FileInfo: file_size=-1, is_dir=False, path='helloworld.txt'>]

In [6]:
client.download_artifacts(run_id, "helloworld.txt", ".")

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/wandent-msft2/code/Users/wandent/3dview/aml/code/mlflow/aml-tracking/helloworld.txt'